In [1]:
pwd

'G:\\내 드라이브\\revive_project\\squrt'

In [8]:
import pandas as pd
import patsy
import numpy as np 
from statsmodels.regression.linear_model import OLS
from scipy import stats
import os
import glob
import matplotlib.pyplot as plt
from numpy import genfromtxt

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import recall_score, f1_score, precision_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics 

from lightgbm import LGBMClassifier
from lightgbm import plot_importance

from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance 

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVC



import warnings

import tensorflow as tf
import datetime
warnings.filterwarnings('ignore')

import funcs
#import funcs_test

%matplotlib inline

In [2]:
right_dir_path = 'G:\\내 드라이브\\revive_project\\squrt\\dataset\\squat'
wrong_dir_path = 'G:\\내 드라이브\\revive_project\\squrt\\dataset\\cheating-손으로까딱'
fake_dir_path = 'G:\\내 드라이브\\revive_project\\squrt\\dataset\\squat'

right_raw_data = funcs.make_right_raw_data(right_dir_path)
wrong_raw_data = funcs.make_wrong_raw_data(wrong_dir_path)
fake_raw_data = funcs.make_fake_raw_data(fake_dir_path)


In [3]:
want_to_make = [right_raw_data, wrong_raw_data, fake_raw_data]

raw_data = np.concatenate(want_to_make, axis =0)

print( ' 정상 데이터 shape : {0} \n 까딱 데이터 shape : {1} \n 1s delayed 데이터 shape : {2} \n 전체 데이터 shape : {3}\n'\
      .format(right_raw_data.shape, wrong_raw_data.shape, fake_raw_data.shape ,raw_data.shape ))

print( '정상 : 비정상(까딱+ 1s delayed) = {0} : {1}'\
      .format(1, round((wrong_raw_data.shape[0]+fake_raw_data.shape[0]) / right_raw_data.shape[0],2)))

 정상 데이터 shape : (733, 123) 
 까딱 데이터 shape : (249, 123) 
 1s delayed 데이터 shape : (657, 123) 
 전체 데이터 shape : (1639, 123)

정상 : 비정상(까딱+ 1s delayed) = 1 : 1.24


In [4]:
batch_size = 20
bases = ['acc_x_','acc_y_','acc_z_','gyr_x_','gyr_y_','gyr_z_']
cols = []
#print('acc_x_' + str(0))
#print('acc_x_' + str(1))

for base in bases:
    for i in range(batch_size):
        cols.append(base+str(round(0.1*i,1)))
        
cols = cols + ['file_name', 'start_sec', 'label']

In [5]:
df = pd.DataFrame(raw_data, columns = cols)

X = df[df.columns[:-3]] 
y = df[df.columns[-1]] 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 , random_state = 0)

y_test = y_test.astype('int')
y_train = y_train.astype('int')

### gpu

In [17]:
with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True)) as sess: 

    with tf.device('/device:GPU:0'):
        
        #실행할 코드
        
        random_matrix = tf.random.uniform(shape=shape, minval=0, maxval=1)
        dot_operation = tf.matmul(random_matrix, tf.transpose(random_matrix))
        sum_operation = tf.reduce_sum(dot_operation)
        # Time the actual runtime of the operations
        start_time = datetime.datetime.now()

        result = sess.run(sum_operation)
        elapsed_time = datetime.datetime.now() - start_time

        print('shape' +": " + str(shape))
        print("elapesed time: " + str (elapsed_time))
        

        # PRINT ELAPSED TIME, SHAPE AND DEVICE USED  

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device

shape: (500, 500)
elapesed time: 0:00:00.026352


In [18]:
with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True)) as sess: 

    with tf.device('/device:GPU:0'):
        
        #실행할 코드
        
        pipe = Pipeline([('classifier' , RandomForestClassifier())])

        # Create param grid.

        param_grid = [
            {'classifier' : [LogisticRegression()],
             'classifier__penalty' : ['l1', 'l2'],
            'classifier__C' : np.logspace(-4, 4, 20),
            'classifier__solver' : ['liblinear']},

            {'classifier' : [RandomForestClassifier()],
             'classifier__max_depth' : list(range(1,12,2)),
             'classifier__min_samples_leaf' : list(range(3,12,2)),
            'classifier__min_samples_split' : list(range(2,12,2)),
            'classifier__n_estimators' : list(range(10,101,10)),   #랜덤 포레스트 안의 결정 트리 갯수 
            'classifier__max_features' : list(range(6,32,5))}   #무작위로 선택할 Feature의 개수 
            ]

        # Create grid search object

        clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, scoring='accuracy'
                       ,verbose=True, n_jobs=-1)

        # Fit on data

        best_clf = clf.fit(X_train, y_train)


        print("tuned hpyerparameters :(best parameters) ",best_clf.best_params_)
        print("train set_accuracy :",best_clf.best_score_)

        # PRINT ELAPSED TIME, SHAPE AND DEVICE USED  

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device

Fitting 5 folds for each of 9040 candidates, totalling 45200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   52.8s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 3168 tasks      | elapsed: 16.6min
[Parallel(n_jobs=-1)]: Done 4018 tasks      | elapsed: 21.0min
[Parallel(n_jobs=-1)]: Done 4968 tasks      | elapsed: 25.9min
[Parallel(n_jobs=-1)]: Done 6018 tasks      | elapsed: 31.2min
[Parallel(n_jobs=-1)]: Done 7168 tasks      | elapsed: 37.2min
[Parallel(n_jobs=-1)]: Done 8418 tasks      | elapsed: 43.7min
[Parallel(n_jobs=-1)]: Done 9768 tasks      | elapsed: 50.6min
[Parallel(n_jobs=-1)]: Done 11218 tasks      

tuned hpyerparameters :(best parameters)  {'classifier': RandomForestClassifier(max_depth=11, max_features=16, min_samples_leaf=9,
                       min_samples_split=4, n_estimators=80), 'classifier__max_depth': 11, 'classifier__max_features': 16, 'classifier__min_samples_leaf': 9, 'classifier__min_samples_split': 4, 'classifier__n_estimators': 80}
train set_accuracy : 0.9115316518155169


In [ ]:
pipe = Pipeline([('classifier' , RandomForestClassifier())])

# Create param grid.

param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 20),
    'classifier__solver' : ['liblinear']},
    
    {'classifier' : [RandomForestClassifier()],
     'classifier__max_depth' : list(range(1,12,2)),
     'classifier__min_samples_leaf' : list(range(3,12,2)),
    'classifier__min_samples_split' : list(range(2,12,2)),
    'classifier__n_estimators' : list(range(10,101,10)),   #랜덤 포레스트 안의 결정 트리 갯수 
    'classifier__max_features' : list(range(6,32,5))}   #무작위로 선택할 Feature의 개수 
]

# Create grid search object

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, scoring='accuracy'
                   ,verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(X_train, y_train)


print("tuned hpyerparameters :(best parameters) ",best_clf.best_params_)
print("train set_accuracy :",best_clf.best_score_)